In [1]:
!nvidia-smi

Sat Dec 18 11:00:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp /content/drive/MyDrive/Suicide_Detection.csv ./

In [5]:
!pip install -q tflite-model-maker --ignore-installed

     |████████████████████████████████| 616 kB 5.5 MB/s 
     |████████████████████████████████| 840 kB 37.6 MB/s 
     |████████████████████████████████| 1.1 MB 33.2 MB/s 
     |████████████████████████████████| 203 kB 46.0 MB/s 
     |████████████████████████████████| 489.6 MB 23 kB/s 
     |████████████████████████████████| 1.2 MB 39.0 MB/s 
     |████████████████████████████████| 4.0 MB 32.0 MB/s 
     |████████████████████████████████| 120 kB 43.9 MB/s 
     |████████████████████████████████| 1.1 MB 41.1 MB/s 
     |████████████████████████████████| 127 kB 46.6 MB/s 
     |████████████████████████████████| 108 kB 50.7 MB/s 
     |████████████████████████████████| 87 kB 6.2 MB/s 
     |████████████████████████████████| 596 kB 37.7 MB/s 
     |████████████████████████████████| 6.4 MB 39.9 MB/s 
     |████████████████████████████████| 15.7 MB 41.2 MB/s 
     |████████████████████████████████| 3.1 MB 30.8 MB/s 
     |████████████████████████████████| 126 kB 47.9 MB/s 
     |██████████

In [1]:
import numpy as np
import os
import pandas as pd
from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.text_classifier import AverageWordVecSpec
from tflite_model_maker.text_classifier import DataLoader
import sklearn

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

In [2]:
df = pd.read_csv('./Suicide_Detection.csv')
df['split'] = np.random.randn(df.shape[0], 1)

msk = np.random.rand(len(df)) <= 0.8

train = df[msk]
test = df[~msk]

In [3]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [5]:
train.shape


(185738, 4)

In [6]:
spec = model_spec.get('mobilebert_classifier')

In [7]:
train_data = DataLoader.from_csv(
      filename='train.csv',
      text_column='text',
      label_column='class',
      model_spec=spec,
      is_training=True)
test_data = DataLoader.from_csv(
      filename='test.csv',
      text_column='text',
      label_column='class',
      model_spec=spec,
      is_training=False)

In [8]:
print(test_data)

In [ ]:
model = text_classifier.create(train_data, model_spec=spec, epochs=7)

3869/3869 [==============================] - 3818s 969ms/step - loss: 0.2505 - test_accuracy: 0.9108
Epoch 2/2
3869/3869 [==============================] - 3753s 969ms/step - loss: 0.0655 - test_accuracy: 0.9778
Epoch 3/3
3869/3869 [==============================] - 3758s 971ms/step - loss: 0.0361 - test_accuracy: 0.9891
Epoch 4/4
3869/3869 [==============================] - 3773s 974ms/step - loss: 0.0186 - test_accuracy: 0.9951
Epoch 5/5
3869/3869 [==============================] - 3762s 972ms/step - loss: 0.0092 - test_accuracy: 0.9977
Epoch 6/6
   2261/Unknown - 2196s 970ms/step - loss: 0.0046 - test_accuracy: 0.9989

In [ ]:
model.export(export_dir='mobilebert/')

In [ ]:
!cp mobilebert/model.tflite /content/drive/MyDrive/model.tflite

In [ ]:
accuracy = model.evaluate_tflite('mobilebert/model.tflite', test_data)
print('TFLite model accuracy: ', accuracy)